In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-2.3.3-bin-hadoop2.7'

In [2]:
import pyspark
findspark.find()

'C:\\spark\\spark-2.3.3-bin-hadoop2.7'

In [3]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("spark_airport").getOrCreate()

In [19]:
midata = spark.read.csv('./train.csv', header = 'true', inferSchema = 'true', sep = ','  )  

In [20]:
midata = midata.dropna(subset=["TotalBsmtSF","OverallQual","1stFlrSF","GrLivArea","GarageArea"])

In [21]:
import pyspark.ml.feature as ft
import pyspark.sql.types as typ
midata = midata.withColumn('YearBuilt', midata['YearBuilt'].cast(typ.DoubleType()))
midata = midata.withColumn('SalePrice', midata['SalePrice'].cast(typ.DoubleType()))
midata = midata.withColumn('TotalBsmtSF', midata['TotalBsmtSF'].cast(typ.DoubleType()))
midata = midata.withColumn('OverallQual', midata['OverallQual'].cast(typ.DoubleType()))
midata = midata.withColumn('1stFlrSF', midata['1stFlrSF'].cast(typ.DoubleType()))
midata = midata.withColumn('GrLivArea', midata['GrLivArea'].cast(typ.DoubleType()))
midata = midata.withColumn('GarageArea', midata['GarageArea'].cast(typ.DoubleType()))

In [22]:
from pyspark.ml import Pipeline
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressionModel, RandomForestRegressor
import pyspark.ml.feature as ft


In [33]:
featuresCreator =VectorAssembler( inputCols=["YearBuilt","TotalBsmtSF","OverallQual","GrLivArea","GarageArea"],outputCol='Independent Feature')

In [34]:
train_data,test_data=midata.randomSplit([0.80,0.20])
    

In [35]:
rf =RandomForestRegressor( labelCol='SalePrice' ,featuresCol='Independent Feature' )

In [36]:
pipeline = Pipeline(stages=[ featuresCreator, rf])

In [37]:
model = pipeline.fit(train_data)

In [38]:
predictions = model.transform(train_data)

In [39]:
predictions.select("prediction", "SalePrice", "Independent Feature").show()

+------------------+---------+--------------------+
|        prediction|SalePrice| Independent Feature|
+------------------+---------+--------------------+
|198413.19789827877| 208500.0|[2003.0,856.0,7.0...|
|213302.48876623073| 223500.0|[2001.0,920.0,7.0...|
|175575.62366555398| 140000.0|[1915.0,756.0,7.0...|
|160014.33393449016| 143000.0|[1993.0,796.0,5.0...|
| 266792.7146801371| 307000.0|[2004.0,1686.0,8....|
| 175403.3011775034| 129900.0|[1931.0,952.0,7.0...|
|133793.02340417303| 129500.0|[1965.0,1040.0,5....|
| 127382.1052183607| 144000.0|[1962.0,912.0,5.0...|
|230907.30297611988| 279500.0|[2006.0,1494.0,7....|
|147003.76386009235| 157000.0|[1960.0,1253.0,6....|
|143106.06888748982| 132000.0|[1929.0,832.0,7.0...|
| 144725.7201597892| 149000.0|[1970.0,1004.0,6....|
|117037.93411291546|  90000.0|[1967.0,0.0,4.0,1...|
| 147032.3038978454| 159000.0|[2004.0,1114.0,5....|
|132108.87055437168| 139000.0|[1958.0,1029.0,5....|
|304415.58861378866| 325300.0|[2005.0,1158.0,8....|
|126809.6888

In [40]:
evaluator = RegressionEvaluator(labelCol='SalePrice', predictionCol ='prediction',metricName = 'rmse')

Error cuadratico medio mide el promedio de los errores al cuadrado, es decir, la diferencia entre el estimador y lo que se estima

In [41]:
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
print("MSE: %.3f" % rmse)

MSE: 30121.998


El coeficiente de determinacion determina la calidad del modelo para Predecir los resultados

In [42]:
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
print("r2: %.3f" %r2)

r2: 0.848
